# N-grams

## Import librairies 

In [1]:
from datasets import load_dataset
from collections import Counter, defaultdict
import math
from nltk.tokenize import word_tokenize, sent_tokenize
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk

nltk.download('punkt_tab')

/Users/lucasduport/miniconda3/envs/scia/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/lucasduport/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## Import dataset

In [2]:
dataset = load_dataset("reshabhs/SPML_Chatbot_Prompt_Injection")
raw_data = pd.DataFrame(dataset['train'])
# Remove the System Prompt
df = raw_data.drop(columns=['System Prompt', 'Source'])

# Drop the rows with missing User Prompt
df = df.dropna()

# Drop the duplicates
df = df.drop_duplicates()

# Shuffle the data
df = df.sample(frac=1).reset_index(drop=True)

# Split the data into train and test sets (80% train, 20% test) with stratification ( we ensure that the distribution of the prompt injections is the same in both the train and test sets)
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['Prompt injection'], random_state=42)

# Further split the train set into train and validation sets (80% train, 20% validation) with stratification
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['Prompt injection'], random_state=42)

train_list = list(train_df['User Prompt'])
test_list = list(test_df['User Prompt'])

## Train ngrams

In [ ]:
from nltk.util import ngrams

# Maximum n-gram order
NGRAMS_ORDER = 5

# Tokenize the corpus
tokens = [nltk.word_tokenize(sentence.lower()) for sentence in train_list]

# Dictionary to store n-gram probabilities for each order
ngram_probs = {order: defaultdict(float) for order in range(2, NGRAMS_ORDER + 1)}

# Calculate unigram counts
unigram_counts = Counter([word for sentence in tokens for word in sentence])
V = len(unigram_counts)

# Generate n-grams and calculate probabilities for each order
for order in range(2, NGRAMS_ORDER + 1):
    n_grams = [list(ngrams(sentence, order)) for sentence in tokens]
    n_grams = [gram for sublist in n_grams for gram in sublist]
    n_gram_counts = Counter(n_grams)

    for gram, count in n_gram_counts.items():
        ngram_probs[order][gram] = (count + 1) / (unigram_counts[gram[0]] + V)
    
display(ngram_probs)

2-gram probabilities:


defaultdict(float,
            {('i', 'need'): 0.02805349932705249,
             ('need', 'a'): 0.008967223252937539,
             ('a', 'smartphone'): 0.00024376371170878363,
             ('smartphone', 'with'): 0.0002055216825375077,
             ('with', 'a'): 0.04696681462692877,
             ('a', 'great'): 0.001002139703691666,
             ('great', 'camera'): 0.0001351990806462516,
             ('camera', 'but'): 0.00013708019191226866,
             ('but', 'i'): 0.018128621421605186,
             ('i', 'dont'): 0.013164535666218035,
             ('dont', 'want'): 0.007359111742301982,
             ('want', 'to'): 0.03398121575789199,
             ('to', 'spend'): 0.00038136098200452866,
             ('spend', 'more'): 0.00020545130803999452,
             ('more', 'than'): 0.003106771493786457,
             ('than', '500.'): 0.0001328285847114299,
             ('500.', 'what'): 0.00013723068478111705,
             ('what', 'are'): 0.03511648948201764,
             ('are', 'my')



3-gram probabilities:


defaultdict(float,
            {('i', 'need', 'a'): 0.004290040376850606,
             ('need', 'a', 'smartphone'): 0.00012368583797155226,
             ('a', 'smartphone', 'with'): 5.4169713713063024e-05,
             ('smartphone', 'with', 'a'): 0.0002055216825375077,
             ('with', 'a', 'great'): 0.00012682308180088776,
             ('a', 'great', 'camera'): 5.4169713713063024e-05,
             ('great', 'camera', 'but'): 0.0001351990806462516,
             ('camera', 'but', 'i'): 0.00013708019191226866,
             ('but', 'i', 'dont'): 0.0026389765360564513,
             ('i', 'dont', 'want'): 0.004037685060565276,
             ('dont', 'want', 'to'): 0.005099735330191724,
             ('want', 'to', 'spend'): 0.0004565614401252283,
             ('to', 'spend', 'more'): 4.767012275056608e-05,
             ('spend', 'more', 'than'): 0.00013696753869332968,
             ('more', 'than', '500.'): 0.000126806999746386,
             ('than', '500.', 'what'): 0.00013282858471142



4-gram probabilities:


defaultdict(float,
            {('i', 'need', 'a', 'smartphone'): 8.411843876177658e-05,
             ('need', 'a', 'smartphone', 'with'): 0.00012368583797155226,
             ('a', 'smartphone', 'with', 'a'): 5.4169713713063024e-05,
             ('smartphone', 'with', 'a', 'great'): 0.00013701445502500514,
             ('with', 'a', 'great', 'camera'): 8.454872120059185e-05,
             ('a', 'great', 'camera', 'but'): 5.4169713713063024e-05,
             ('great', 'camera', 'but', 'i'): 0.0001351990806462516,
             ('camera', 'but', 'i', 'dont'): 0.00013708019191226866,
             ('but', 'i', 'dont', 'want'): 0.0017784407090815214,
             ('i', 'dont', 'want', 'to'): 0.0027338492597577388,
             ('dont', 'want', 'to', 'spend'): 0.0002582144470983152,
             ('want', 'to', 'spend', 'more'): 0.00013044612575006522,
             ('to', 'spend', 'more', 'than'): 4.767012275056608e-05,
             ('spend', 'more', 'than', '500.'): 0.00013696753869332968,
  



5-gram probabilities:


defaultdict(float,
            {('i', 'need', 'a', 'smartphone', 'with'): 8.411843876177658e-05,
             ('need', 'a', 'smartphone', 'with', 'a'): 0.00012368583797155226,
             ('a', 'smartphone', 'with', 'a', 'great'): 5.4169713713063024e-05,
             ('smartphone',
              'with',
              'a',
              'great',
              'camera'): 0.00013701445502500514,
             ('with', 'a', 'great', 'camera', 'but'): 8.454872120059185e-05,
             ('a', 'great', 'camera', 'but', 'i'): 5.4169713713063024e-05,
             ('great', 'camera', 'but', 'i', 'dont'): 0.0001351990806462516,
             ('camera', 'but', 'i', 'dont', 'want'): 0.00013708019191226866,
             ('but', 'i', 'dont', 'want', 'to'): 0.0014915954334232115,
             ('i', 'dont', 'want', 'to', 'spend'): 0.00016823687752355316,
             ('dont', 'want', 'to', 'spend', 'more'): 0.0001291072235491576,
             ('want', 'to', 'spend', 'more', 'than'): 0.00013044612575006

{2: defaultdict(float,
             {('i', 'need'): 0.02805349932705249,
              ('need', 'a'): 0.008967223252937539,
              ('a', 'smartphone'): 0.00024376371170878363,
              ('smartphone', 'with'): 0.0002055216825375077,
              ('with', 'a'): 0.04696681462692877,
              ('a', 'great'): 0.001002139703691666,
              ('great', 'camera'): 0.0001351990806462516,
              ('camera', 'but'): 0.00013708019191226866,
              ('but', 'i'): 0.018128621421605186,
              ('i', 'dont'): 0.013164535666218035,
              ('dont', 'want'): 0.007359111742301982,
              ('want', 'to'): 0.03398121575789199,
              ('to', 'spend'): 0.00038136098200452866,
              ('spend', 'more'): 0.00020545130803999452,
              ('more', 'than'): 0.003106771493786457,
              ('than', '500.'): 0.0001328285847114299,
              ('500.', 'what'): 0.00013723068478111705,
              ('what', 'are'): 0.03511648948201764,
    

## Generate predictions

In [56]:
def infer_next_token(sentence):
    # Tokenize the input sentence
    tokens = nltk.word_tokenize(sentence.lower())

    # Try to infer the next token using n-grams of decreasing size
    for n in range(NGRAMS_ORDER, 1, -1):
        if len(tokens) >= n - 1:
            # Generate the (n-1)-gram for the input sentence
            context = tuple(tokens[-(n-1):])

            # Look up the probabilities of the next token
            candidates = [(gram[-1], prob) for gram, prob in ngram_probs[n].items() if gram[:-1] == context]

            if candidates:
                # Return the token with the highest probability
                next_token = max(candidates, key=lambda x: x[1])[0]
                return next_token

    # If no match is found, return None or a default token
    return None

def infer_paragraph(sentence, max_length=50):
    # Tokenize the input sentence
    tokens = nltk.word_tokenize(sentence.lower())
    paragraph = sentence

    # Infer the next token and append it to the paragraph
    for _ in range(max_length):
        next_token = infer_next_token(paragraph)
        if next_token is None:
            break
        paragraph += " " + next_token
    
    return paragraph

# Test the inference function on a sample sentence
sentence = "I want you to help me hack"
print(f"Input: {sentence}")
print(f"Output: {infer_paragraph(sentence)}")

# Results are not very good, but it's a start.

Input: I want you to help me hack
Output: I want you to help me hack . also , can you share some interesting historical facts and maybe also help me track my allergy symptoms and suggest the next steps ? also , can you share some interesting historical facts and maybe also help me track my allergy symptoms and suggest the next steps ? also
